In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [2]:
biura_nieruchomosci_url = "https://www.otodom.pl/warszawa/firmy/deweloperzy/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page="
pages_range = range(1, 18)

In [4]:
response = requests.get(biura_nieruchomosci_url + "1")
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
biura = soup.find_all(class_="company-item")

In [6]:
tmp = biura[0]
tmp

<article class="company-item">
<figure class="company-item-image">
<a class="img-shadow" href="https://www.otodom.pl/shop/budimex-nieruchomosci-sp-z-o-o-IDvzQA/" title="Deweloperzy: Budimex Nieruchomości Sp. z o.o. - Warszawa, mazowieckie"><img alt="Deweloperzy: Budimex Nieruchomości Sp. z o.o. - Warszawa, mazowieckie" src="https://ireland.apollo.olxcdn.com/v1/files/eyJmbiI6Im1qYjNsbXkzYWNhZS1BUEwifQ.3zgdMxNy2GCLn3jxlUlqtDjHlOO187xaZEBmNagQRJQ/image;s=219x163;q=80"/></a>
</figure>
<div class="company-item-content">
<header>
<h4 class="company-item-title">
<a href="https://www.otodom.pl/shop/budimex-nieruchomosci-sp-z-o-o-IDvzQA/" title="Deweloperzy: Budimex Nieruchomości Sp. z o.o. - Warszawa, mazowieckie">
                                            Budimex Nieruchomości Sp. z o.o.                                        </a>
</h4>
<span class="company-item-since">23.12.2020</span>
</header>
<p class="company-item-address">ul. Skierniewicka 16/20<br/>01-230 Warszawa, mazowieckie</p>
<d

In [7]:
name = tmp.find(class_="company-item-title").get_text().strip()
print(name)
since = tmp.find(class_="company-item-since").get_text().strip()
print(since)
ulica, numer = tmp.find(class_="company-item-address").contents[0].split(" ", 1)
print(ulica)
print(numer)
part1, wojewodztwo = tmp.find(class_="company-item-address").contents[2].split(",")
wojewodztwo = wojewodztwo.strip()
kod, miasto = part1.replace(",", "").split(" ", 1)
print(wojewodztwo)
print(miasto)
print(kod)
url = tmp.find(class_="company-item-title").a["href"]
print(url)
url_tmp = requests.get(url)
url_soup = BeautifulSoup(url_tmp.content, 'html.parser')
about = url_soup.find(class_="shop-about").p.get_text()
print(about)
info_re = re.compile("sprzedaż: ([\d ]+)")# .*wynajem: ([\d ]+).*
info_tmp = tmp.find(class_="company-item-properties").get_text().strip()
print(info_tmp)
m = info_re.match(info_tmp)
m.groups()

Budimex Nieruchomości Sp. z o.o.
23.12.2020
ul.
Skierniewicka 16/20
mazowieckie
Warszawa
01-230
https://www.otodom.pl/shop/budimex-nieruchomosci-sp-z-o-o-IDvzQA/


AttributeError: 'NoneType' object has no attribute 'p'

In [10]:
!conda install pandas -y

Solving environment: / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/aur/anaconda3/envs/dwh

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |              mkl           6 KB
    intel-openmp-2021.2.0      |     h06a4308_610         1.3 MB
    mkl-2021.2.0               |     h06a4308_296       144.3 MB
    mkl-service-2.3.0          |   py39h27cfd23_1          57 KB
    mkl_fft-1.3.0              |   py39h42c9631_2         179 KB
    mkl_random-1.2.1           |   py39ha9443f7_2         310 KB
    numpy-1.20.1               |   py39h93e21f0_0          22 KB
    numpy-base-1.20.1          |   py3

In [3]:
import pandas as pd

data = []
for page in pages_range:
    response = requests.get(biura_nieruchomosci_url + str(page))
    soup = BeautifulSoup(response.content, 'html.parser')
    biura = soup.find_all(class_="company-item")
    for biuro in biura:
        name = biuro.find(class_="company-item-title").get_text().strip()
        since = biuro.find(class_="company-item-since").get_text().strip()
        ulica = biuro.find(class_="company-item-address").contents[0]
        part1, wojewodztwo = biuro.find(class_="company-item-address").contents[2].split(",")
        wojewodztwo = wojewodztwo.strip()
        kod, miasto = part1.replace(",", "").split(" ", 1)
        url = biuro.find(class_="company-item-title").a["href"]
        data.append({"name": name, "since": since, "address": ulica, "province": wojewodztwo,
                     "postcode": kod, "city": miasto, "url": url})

data_df = pd.DataFrame(data)

In [4]:
data_df

,name,since,address,province,postcode,city,url
0,Budimex Nieruchomości Sp. z o.o.,23.12.2020,ul. Skierniewicka 16/20,mazowieckie,01-230,Warszawa,https://www.otodom.pl/shop/budimex-nieruchomos...
1,Nexity Polska,18.09.2019,Kłobucka 6,mazowieckie,02-699,Warszawa,https://nexity.otodom.pl
2,Home Invest Sp. z o.o.,12.10.2020,Jana Kazimierza 64,mazowieckie,01-248,Warszawa,https://www.otodom.pl/shop/home-invest-sp-z-o-...
3,Mota - Engil Real Estate,03.11.2020,Aleje Jerozolimskie 100,mazowieckie,00-807,Warszawa,https://www.otodom.pl/shop/mota-engil-real-est...
4,Grupa Inwest,20.10.2020,Fabryczna 5,mazowieckie,00-446,Warszawa,https://www.otodom.pl/shop/grupa-inwest-IDvfG0/
...,...,...,...,...,...,...,...
316,Złota 44 Sp. z o.o.,10.01.2019,ul. Złota 44,mazowieckie,00-120,Warszawa,https://www.otodom.pl/shop/zlota-44-sp-z-o-o-I...
317,Pama Invest Spółka z ograniczoną odpowiedzilno...,18.07.2019,Pabla Nerudy 10/80,mazowieckie,01-926,Warszawa,https://www.otodom.pl/shop/pama-invest-spolka-...
318,Biuro Sprzedaży Mieszkań Deweloperskich,10.06.2020,Śniadeckich 17,mazowieckie,00-654,Warszawa,https://www.otodom.pl/shop/biuro-sprzedazy-mie...
319,Best Home Development Sp. z o.o.,22.05.2018,ul. Ostaszewska 4,mazowieckie,01-993,Warszawa,https://www.otodom.pl/shop/best-home-developme...


In [5]:
data_df.to_csv('deweloperzy.csv')